<a href="https://www.kaggle.com/code/jorgeluiscruzquispe/precipitation-chirps-gpm?scriptVersionId=161447711" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <center>DESCARGA Y COMPARACIÓN DE PRECIPITACIÓN SATELITAL DE CHIRPS Y GPM</center>
---

In [ ]:
!pip install geemap

In [ ]:
import geemap.foliumap as geemap
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
#Centrar imagen
Map = geemap.Map()

# Especificar la cuenca
cuenca = "Cuenca Santa"

# estaciones}

coordenadas = [[-78.2930, -8.1215],
               [-78.2083, -9.4337],
               [-78.1500, -8.1000],
               [-77.9542, -8.1916],
               [-77.7754, -9.5352],
               [-77.6500, -8.5667],
               [-77.4538, -9.7292],
               [-77.3489, -10.1578],
               [-77.2333, -9.8833]]

stations = ee.Geometry.MultiPoint(coordenadas)


#Load pp GPM
chirp = ee.ImageCollection(ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")\
                         .select(['precipitation'])\
                         .filterDate('2010-01-01', '2011-12-31'))
                         
chirp_v6 = chirp.sum()

# load vector cuenca
basin = ee.FeatureCollection('users/mportuguez/CuencasPeru')

# Seleccionar cuenca
selected = basin.filter(ee.Filter.eq('NOMBRE', cuenca))
roi = selected.geometry()

chirp_corte = chirp_v6.clip(roi)

# coleccion visibildad - vist parameter
ColorCHIRP = {
    'min' : 0.0,
    'max' : 1700.0,
    'palette': ['ff7c2f', 'fff823', '1eff10', '2dffff', '293aff'],
    'opacity': 0.5
}

#centrar y zoom
Map.centerObject(chirp_corte, 8)

# visualizar imagen
Map.addLayer(chirp_corte, ColorCHIRP, 'PP CHIRPS')
Map.addLayer(ee.Image().paint(selected, 0, 2), {'palette': 'red'}, cuenca)
Map.addLayer(stations, {"color": "orange"}, 'Estaciones')
Map

In [ ]:
def create_reduce_region_function(geometry,
                                  reducer = ee.Reducer.mean(),
                                  scale = 1000,
                                  crs = 'EPSG:4326',
                                  bestEffort = True,
                                  maxPixels = 1e13,
                                  tileScale = 4):

  def reduce_region_function(img):
    stat = img.reduceRegion(
        reducer = reducer,
        geometry = geometry,
        scale = scale,
        crs = crs,
        bestEffort = bestEffort,
        maxPixels = maxPixels,
        tileScale = tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer = ee.Reducer.toList().repeat(prop_names.size()),
      selectors = prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

## <center>Descarga precipitación de data Chirps</center>
---

In [ ]:
fecha_inicial = '2010-01-01'; fecha_final = '2010-12-31'

n_c = 0

n2 = 1

resultados_chirps = pd.DataFrame()

for latitude, longitude in coordenadas:

    point = ee.Geometry.Point([latitude, longitude])
 

    chirps =  ee.ImageCollection(ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")\
                             .select(['precipitation'])\
                             .filterDate(fecha_inicial, fecha_final))

    puntos_red = create_reduce_region_function(geometry = point, reducer = ee.Reducer.first(), scale = 5000, crs = 'EPSG:4326')

    chirps_red = (ee.FeatureCollection(chirps.map(puntos_red)).filter(ee.Filter.notNull(chirps.first().bandNames())))

    puntos_chirps = fc_to_dict(chirps_red).getInfo()
    
    resultados_chirps["estacion "+ str(n2)] = puntos_chirps["precipitation"]
    
    n2 = n2 + 1

    resultados_chirps['Fecha'] = pd.to_datetime(puntos_chirps["system:index"], format='%Y%m%d')
        
    resultados_chirps.set_index('Fecha', inplace=True)
    
# resultados_chirps.to_csv("resultados.csv")
    
resultados_chirps

In [ ]:
resultados_chirps.describe().T

In [ ]:
plt.figure(figsize = (20, 5))

plt.title("Hietograma de las estaciones de data chirps")

colors = ["black", "blue", "orange", "green", "yellow", "pink", "red", "purple", "brown"]

n = 1

for i in range(1, 10):
    plt.plot(resultados_chirps.index, 
             resultados_chirps["estacion " + str(i)], 
             color = colors[i - 1], 
             alpha = 0.5, 
             label = "estacion "+ str(i))

plt.legend()

plt.grid(color = "silver", alpha = 0.4)

plt.ylabel("precipitación $(mm)$")

plt.xlabel("tiempo $(días)$")

plt.show()

## <center>Descarga precipitación de data GPM</center>
---

In [ ]:
fecha_inicial = '2010-01-01'; fecha_final = '2010-12-31'

resultados_gpm = pd.DataFrame()

n_c = 0

n3 = 1

for latitude, longitude in coordenadas:

    point = ee.Geometry.Point([latitude, longitude])
 
    gpm =  ee.ImageCollection(ee.ImageCollection('NASA/GPM_L3/IMERG_V06')\
                             .select(['precipitationCal'])\
                             .filterDate(fecha_inicial, fecha_final))

    puntos_red = create_reduce_region_function(geometry=point, reducer=ee.Reducer.first(), scale = 5000, crs = 'EPSG:4326')
    
    gpm_red = (ee.FeatureCollection(gpm.map(puntos_red)).filter(ee.Filter.notNull(gpm.first().bandNames())))
    
    puntos_gpm = fc_to_dict(gpm_red).getInfo()
    
    resultados_gpm["estacion "+ str(n3)] = puntos_gpm["precipitationCal"]
    
    n3 = n3 + 1
    
    resultados_gpm['Fecha'] = pd.to_datetime(puntos_gpm['system:index'], format='%Y%m%d%H%M%S')

    resultados_gpm.set_index('Fecha', inplace=True)
    
resultados_gpm = resultados_gpm.resample('d').sum()

# resultados_gpm.to_csv("resultados_gpm.csv")
    
resultados_gpm

In [ ]:
resultados_gpm.describe().T

In [ ]:
plt.figure(figsize = (20, 5))

plt.title("Hietograma de las estaciones de data gpm")

colors = ["black", "blue", "orange", "green", "yellow", "pink", "red", "purple", "brown"]

n = 1

for i in range(1, 10):
    plt.plot(resultados_gpm.index, 
             resultados_gpm["estacion " + str(i)], 
             color = colors[i - 1], 
             alpha = 0.5, 
             label = "estacion "+ str(i))

plt.legend()

plt.grid(color = "silver", alpha = 0.4)

plt.ylabel("precipitación $(mm)$")

plt.xlabel("tiempo $(días)$")

plt.show()

## <center>Comparación para una estación entre data chirps y gpm</center>
---

In [ ]:
estacion_fijo = str(9)

plt.figure(figsize = (20, 5))

plt.title("Hietograma de las estaciones de data CHIRPS y GPM")

plt.plot(resultados_gpm.index, 
         resultados_gpm["estacion "+ estacion_fijo], "r-",
         alpha = 0.5, 
         label = "estacion "+ estacion_fijo + " chirps")

plt.plot(resultados_chirps.index, 
         resultados_chirps["estacion "+ estacion_fijo], "g-", 
         alpha = 0.5, 
         label = "estacion "+ estacion_fijo + " gpm")

plt.legend()

plt.ylabel("precipitación $(mm)$")

plt.xlabel("tiempo $(días)$")

plt.grid(color = "silver", alpha = 0.4)

plt.show()

## <center>Graficos de dispersión de la data chirps y gpm</center>
---

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))

N = 0

for i1 in range(0, 3):
    for i2 in range(0, 3):
        N += 1
        axes[i1, i2].plot(resultados_chirps[f"estacion {str(N)}"], resultados_gpm[f"estacion {str(N)}"], "bo", alpha = 0.5)
        axes[i1, i2].grid(c = "silver", lw = 0.75)
        axes[i1, i2].set_title(f"Estación {str(N)}")
        axes[i1, i2].set_xlabel("data chirps")
        axes[i1, i2].set_ylabel("data gpm")
        m, b, r, p, std = stats.linregress(resultados_chirps[f"estacion {str(N)}"], resultados_gpm[f"estacion {str(N)}"])
        axes[i1, i2].plot(resultados_chirps[f"estacion {str(N)}"], 
                          b + m * resultados_chirps[f"estacion {str(N)}"], 
                          color='red', 
                          label=f'$f(x) = {m:.2f}x + {b:.2f}$', 
                          alpha = 0.7, 
                          linestyle='-', 
                          linewidth=1.2)
        
        axes[i1, i2].legend([f'$f(x) = {m:.2f}x + {b:.2f}$', 
                             f"$R^2 = {(r**2):.2f}$"], 
                            loc = "upper right", 
                            fontsize = "small")
        
plt.suptitle('Graficos de dispersión entre datos de precipitación de data gpm y chirps')
plt.tight_layout()  
plt.show()